# Essentially same as otbn_find_bits.ipynb but streamlined for 100M captures.

In [ ]:
import numpy as np
wave = np.load('waves_p256_100M_2s.npy')
#wave = np.load('waves_p256_100M_2s_12bits.npy')
#wave = np.load('waves_p256_100M_2s_12bits830.npy')
#wave = np.load('waves_p256_100M_2s_12bitsf0c.npy')

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq 
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=9):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

filtered_wave = butter_highpass_filter(wave, 6e6, 100e6) # for NON-streamed 100M capture

### optional, if we need to plot to understand why we're not finding good bit times:

In [ ]:
#samples = len(waves[0])
samples = 600000
base = 0

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

wf = datashade(hv.Curve(filtered_wave[base:base+samples]), cmap=['black'])
(wf).opts(width=2000, height=600)

### p384 alignment method:

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
mfw = moving_average(np.abs(filtered_wave), 3000)

In [ ]:
len(mfw)

In [ ]:
samples = 600000
base = 0

mwf = datashade(hv.Curve(mfw[base:base+samples]), cmap=['black'])
mwf.opts(width=2000, height=600)

In [ ]:
base = 0
samples = len(filtered_wave)
from scipy.signal import find_peaks
peaks, _ = find_peaks(-mfw[base:base+samples], distance=30000)

In [ ]:
len(peaks), peaks

In [ ]:
bit_starts3 = peaks[1:]

In [ ]:
bit_starts3

In [ ]:
deltas = []
good_deltas = []
good_bits = 0
for i in range(len(bit_starts3)-2):
    delta = bit_starts3[i+1] - bit_starts3[i]
    deltas.append(delta)
    print(delta, end='')
    if 32000 < delta < 32300:
        good_bits += 1
        good_deltas.append(delta)
        print()
    else:
        print(' oops!')
    

In [ ]:
good_bits

In [ ]:
hv.Curve(good_deltas).opts(width=2000, height=900)

In [ ]:
duration = int(np.average(good_deltas))
duration, np.average(good_deltas), max(good_deltas)-min(good_deltas)

In [ ]:
bbstarts = []
for i in range(256):
    bbstarts.append(42970 + i*32153)

# Superimpose all the bits!
Plot overlayed bit traces to visualize alignment and guess at success of time extraction:

In [ ]:
bit_starts = bit_starts3[:256]
#bit_starts = bbstarts

bits = []
bit_size = bit_starts[1] - bit_starts[0]
for start in bit_starts:
    bits.append(filtered_wave[start:start+bit_size])

In [ ]:
len(bits)

In [ ]:
duration

In [ ]:
# Can plot all the bits, but it's slow:
#numbits = len(bits)
#duration = 1000

duration = 32152
numbits = 4

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

xrange = range(duration)

from operator import mul
from functools import reduce

curves = [hv.Curve(zip(xrange, filtered_wave[bit_starts[i]:bit_starts[i]+duration])) for i in range(numbits)]
#curves = [hv.Curve(zip(xrange, filtered_wave[bbstarts[i]:bbstarts[i]+duration])) for i in range(numbits)]

reduce(mul, curves).opts(width=2000, height=900)

## Now try resync:

In [ ]:
import chipwhisperer.analyzer.preprocessing as preprocess
resync = preprocess.ResyncDTW()

In [ ]:
import fastdtw as fastdtw
def align_traces(N, r, ref, trace, cython=True):
    #try:
    if cython:
        # cython version can't take numpy.memmap inputs, so we convert them to arrays:
        aref = np.array(list(ref))
        atrace = np.array(list(trace))
        dist, path = fastdtw.fastdtw(aref, atrace, radius=r, dist=None)
    else:
        dist, path = old_dtw(ref, trace, radius=r, dist=None)
    #except:
    #    return None
    px = [x for x, y in path]
    py = [y for x, y in path]
    n = [0] * N
    s = [0.0] * N
    for x, y in path:
        s[x] += trace[y]
        n[x] += 1

    ret = [s[i] / n[i] for i in range(N)]
    return ret


In [ ]:
ref = bits[0]
target = filtered_wave[bit_starts[1]:bit_starts[1]+duration]
from tqdm.notebook import tnrange

realigns = [ref]
for b in tnrange(1,256):
    target = bits[b]
    realigns.append(np.asarray(align_traces(N=len(ref), r=3, ref=ref, trace=target)))


In [ ]:
#numbits = len(bits)
numbits = 40

#curves = [hv.Curve(zip(xrange, realigns[i])) for i in range(numbits)]
curves = [hv.Curve(zip(xrange, realigns[i])) for i in range(128,160)]
reduce(mul, curves).opts(width=2000, height=900)

In [ ]:
b0 = hv.Curve(ref)
b1 = hv.Curve(target)
re = hv.Curve(realigned)
#(b0 * b1 * re).opts(width=2000, height=900)
#(b0 * b1).opts(width=2000, height=900)
(b0 * re).opts(width=2000, height=900)

## Original approach:

In [ ]:
def contiguous_regions(condition):
    """Finds contiguous True regions of the boolean array "condition". Returns
    a 2D array where the first column is the start index of the region and the
    second column is the end index."""

    # Find the indicies of changes in "condition"
    d = np.diff(condition.astype(int))
    idx, = d.nonzero() 

    # We need to start things after the change in "condition". Therefore, 
    # we'll shift the index by 1 to the right.
    idx += 1

    if condition[0]:
        # If the start of condition is True prepend a 0
        idx = np.r_[0, idx]

    if condition[-1]:
        # If the end of condition is True, append the length of the array
        idx = np.r_[idx, condition.size] # Edit

    # Reshape the result into two columns
    idx.shape = (-1,2)
    return idx


### Find runs of samples below threshold value:
(keep only runs that are long enough)

In [ ]:
# for 100M NOT streamed:
THRESHOLD = 0.015
MIN_RUN_LENGTH = 60 # default for the 128 1's / 128 0's
#MIN_RUN_LENGTH = 40

STOP=len(filtered_wave)
#STOP=360000
condition = np.abs(filtered_wave[:STOP]) < THRESHOLD

# Print the start and stop indices of each region where the absolute 
# values of x are below 1, and the min and max of each of these regions
results = contiguous_regions(condition)
#print(len(results))
goods = results[np.where(results[:,1] - results[:,0] > MIN_RUN_LENGTH)]
print(len(goods))

In [ ]:
# to help debug:
last_stop = 0
for g in goods:
    start = g[0]
    stop = g[1]
    l = stop-start
    delta = start - last_stop
    if 13000 < delta < 18000:
        stat = 'ok'
    else:
        stat = 'OOOOPS?!?'
    print('%8d %8d %8d %8d %s' % (l, delta, start, stop, stat))
    last_stop = stop

### Use these runs to guess at bit start times:

In [ ]:
raw_starts = []
for i in range(1, len(goods), 2):
    raw_starts.append(goods[i][1])

In [ ]:
raw_starts[:12]

In [ ]:
duration = raw_starts[1] - raw_starts[0]
print(duration)

### Now we make the bit start times more accurate by using the single isolated large peak that's about 650 samples in:
hmm, not sure if this actually improves the results...

In [ ]:
wstart = 500
wend = 700

#wstart = 1550
#wend = 1620

base = np.argmax(filtered_wave[raw_starts[0]+wstart:raw_starts[0]+wend])
bit_starts = [raw_starts[0]]
for s in raw_starts[1:]:
    loc = np.argmax(filtered_wave[s+wstart:s+wend])
    offset = base-loc
    #print(offset)
    bit_starts.append(s + offset)

In [ ]:
len(raw_starts), len(bit_starts)

In [ ]:
for b in range(11):
    delta = raw_starts[b+1] - raw_starts[b]
    print(delta, end='')
    if not 31000 < delta < 33000:
        print(' Ooops!')
    else:
        print()

# What if we use the SAD approach to find bits instead?

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook

output_notebook(INLINE)

samples = 120000
xrange = range(samples)
S = figure(width=2000, height=900)
S.line(xrange, filtered_wave[:samples], color='blue')

In [ ]:
show(S)

In [ ]:
#base = 45973
#base = 43257
base = 45067

#cycles = 32150 # full bit
#cycles = 32150//2 # half bit
cycles = 2000 # something short
#cycles = 80000 # *more* than one bit

refbit = filtered_wave[base:base+cycles]

from tqdm.notebook import tnrange
diffs = []
for i in tnrange(78000, 500000):
    diffs.append(np.sum(abs(refbit - filtered_wave[i:i+len(refbit)])))

In [ ]:
base + 31350

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

datashade(hv.Curve(diffs)).opts(width=2000, height=900)

# Average 'one' and 'zero'

In [ ]:
duration

In [ ]:
#starts = raw_starts
#starts = bit_starts
starts = bit_starts3[:256]

In [ ]:
# f0c: 1111_0000_1111

In [ ]:
avg_trace = np.zeros(duration)
avg_ones = np.zeros(duration)
avg_zeros = np.zeros(duration)

for i, start in enumerate(starts[:12]):
    avg_trace += filtered_wave[start:start+duration]
    #if i < 6:
    if i < 4 or i > 7:
        avg_ones += filtered_wave[start:start+duration]
    #elif i < 12:
    elif 3 < i < 8:
        avg_zeros += filtered_wave[start:start+duration]

avg_trace /= 12 #len(bit_starts)
#avg_ones /= 6 #len(bit_starts)/2
#avg_zeros /= 6 #len(bit_starts)/2

avg_ones /= 8 #len(bit_starts)/2
avg_zeros /= 4 #len(bit_starts)/2


In [ ]:
for b in range(10):
    print(len(realigns[b]))

In [ ]:
duration = 32151
avg_trace = np.zeros(duration)
avg_ones = np.zeros(duration)
avg_zeros = np.zeros(duration)

for i in range(256):
    avg_trace += realigns[i]
    if i < 128:
        avg_ones += realigns[i]
    else:
        avg_zeros += realigns[i]

avg_trace /= 256
avg_ones /= 128
avg_zeros /= 128


In [ ]:
# what if we don't realign?
duration = 32151
avg_trace = np.zeros(duration)
avg_ones = np.zeros(duration)
avg_zeros = np.zeros(duration)

for i in range(256):
    avg_trace += bits[i]
    if i < 128:
        avg_ones += bits[i]
    else:
        avg_zeros += bits[i]

avg_trace /= 256
avg_ones /= 128
avg_zeros /= 128


In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

xrange = range(duration)

cavg_all = datashade(hv.Curve(avg_trace), cmap=['black'])
cavg_ones = datashade(hv.Curve(avg_ones), cmap=['blue'])
cavg_zeros = datashade(hv.Curve(avg_zeros), cmap=['green'])

cdiff = datashade(hv.Curve((avg_ones - avg_zeros)), cmap=['red'])

#(cavg_all * cavg_ones * cavg_zeros).opts(width=2000, height=900)
#(cdiff * cavg_all).opts(width=2000, height=600)
#(cavg_ones*cavg_zeros).opts(width=2000, height=600)
(cavg_zeros*cavg_ones).opts(width=2000, height=600)

In [ ]:
(cdiff).opts(width=2000, height=600)

In [ ]:
np.average(avg_ones), np.average(avg_zeros)

In [ ]:
np.sum(abs(avg_ones)) / np.sum(abs(avg_zeros))

### attack using just the sum of the power trace segment:

In [ ]:
scores = []
#for b in bit_starts:
for b in raw_starts:
    scores.append(np.sum(abs(filtered_wave[b:b+duration])))

In [ ]:
cscores = hv.Curve(scores[:12])
(cscores).opts(width=2000, height=600)

### attack using markers:

In [ ]:
markers = np.where((avg_ones - avg_zeros) > 0.01)[0]
#markers = np.where(abs(avg_ones - avg_zeros) > 0.005)[0]

In [ ]:
len(markers)

In [ ]:
markers

In [ ]:
scores = []
for b in starts:
    score = 0
    for marker in markers:
        #score += abs(filtered_wave[b + marker])
        score += filtered_wave[b + marker]
    scores.append(score)
    

In [ ]:
cscores = hv.Curve(scores)
(cscores).opts(width=2000, height=600)

In [ ]:
scores = []
for b in range(256):
    score = 0
    for marker in markers:
        score += abs(realigns[b][marker])
    scores.append(score)

In [ ]:
scores = []
for b in range(256):
    score = 0
    for marker in markers:
        score += bits[b][marker]
    scores.append(score)

In [ ]:
scores = []
for b in range(256):
    score = 0
    for m in range(18000,19200):
        score += abs(bits[b][m])
    scores.append(score)

In [ ]:
np.average(scores[:128]), np.average(scores[128:])

In [ ]:
np.average(scores[:10])

In [ ]:
np.average(scores[128:138])

In [ ]:
scores[128:138]

In [ ]:
max(scores), min(scores)